<a href="https://colab.research.google.com/github/khatgarhaastha/WandbExperiment/blob/main/AasthaKhatgarh_Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:19:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [9]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:19:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [10]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [11]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:19:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [12]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:19:31] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [13]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:19:34] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [14]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [15]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [16]:
!pip install wandb

In [17]:
import wandb

wandb.login()

from sklearn.model_selection import GridSearchCV

wandb.init(
    # set the wandb project where this run will be logged
    project="Week 4",
    config = {
        "Logging": "RMSE ERROR",
        "MODEL TYPE": "XGBOOST",
        "Hyper Parameter": ["Learning Rate", "Max Depth"],

    }
)

learning_rates = [0.01, 0.1, 1, 10]
max_depths = [5,6]

for learning_ in learning_rates:
	for max_depth in max_depths:
		params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "learning_rate" : learning_, "max_depth": max_depth}
		model = xgb.train(
		params=params,
		dtrain=dtrain,
		num_boost_round=n,
		evals=evals
		)

		predictions = model.predict(dtest)
		rmse = mean_squared_error(y_test, predictions, squared=False)

		wandb.log({"Learning Rate": learning_, "RMSE_Loss": rmse, "max_depth" : max_depth})




<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khatgarh-aastha (khatgarh-aastha0014). Use `wandb login --relogin` to force relogin


[0]	train-rmse:3952.55708	validation-rmse:3949.80754
[1]	train-rmse:3915.55510	validation-rmse:3912.78792
[2]	train-rmse:3878.97544	validation-rmse:3876.12914
[3]	train-rmse:3842.78846	validation-rmse:3839.85949
[4]	train-rmse:3806.98393	validation-rmse:3803.95353
[5]	train-rmse:3771.56190	validation-rmse:3768.49003
[6]	train-rmse:3736.51203	validation-rmse:3733.35154
[7]	train-rmse:3701.80275	validation-rmse:3698.43428
[8]	train-rmse:3667.46018	validation-rmse:3663.84118
[9]	train-rmse:3633.44516	validation-rmse:3629.59855
[10]	train-rmse:3599.78512	validation-rmse:3595.71835
[11]	train-rmse:3566.52404	validation-rmse:3562.25174
[12]	train-rmse:3533.57538	validation-rmse:3529.06934
[13]	train-rmse:3500.98543	validation-rmse:3496.25290
[14]	train-rmse:3468.73912	validation-rmse:3463.77791
[15]	train-rmse:3436.86308	validation-rmse:3431.70991
[16]	train-rmse:3405.27079	validation-rmse:3399.97296
[17]	train-rmse:3374.01820	validation-rmse:3368.58308
[18]	train-rmse:3343.00014	validation-

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:25:02] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[21]	train-rmse:3251.98019	validation-rmse:3245.93704
[22]	train-rmse:3222.15696	validation-rmse:3215.98916
[23]	train-rmse:3192.74197	validation-rmse:3186.41546
[24]	train-rmse:3163.69066	validation-rmse:3157.17789
[25]	train-rmse:3134.77831	validation-rmse:3128.24724
[26]	train-rmse:3106.31856	validation-rmse:3099.66496
[27]	train-rmse:3078.12726	validation-rmse:3071.44546
[28]	train-rmse:3050.06496	validation-rmse:3043.48244
[29]	train-rmse:3022.50633	validation-rmse:3015.84312
[30]	train-rmse:2995.08799	validation-rmse:2988.39582
[31]	train-rmse:2967.90551	validation-rmse:2961.31213
[32]	train-rmse:2941.11226	validation-rmse:2934.40841
[33]	train-rmse:2914.50049	validation-rmse:2907.85566
[34]	train-rmse:2888.38305	validation-rmse:2881.64791
[35]	train-rmse:2862.48281	validation-rmse:2855.53057
[36]	train-rmse:2836.73166	validation-rmse:2829.75053
[37]	train-rmse:2811.26062	validation-rmse:2804.27246
[38]	train-rmse:2786.05733	validation-rmse:2779.06548
[39]	train-rmse:2761.25391	v

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:26:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:26:05] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[11]	train-rmse:1078039840585116.50000	validation-rmse:1077879103871953.50000
[12]	train-rmse:9692923326102616.00000	validation-rmse:9691302230233348.00000
[13]	train-rmse:81040172322154832.00000	validation-rmse:81146060464305712.00000
[14]	train-rmse:688450974443589632.00000	validation-rmse:687322867045399936.00000
[15]	train-rmse:4305434412174650880.00000	validation-rmse:4312127161504396800.00000
[16]	train-rmse:inf	validation-rmse:inf
[17]	train-rmse:inf	validation-rmse:inf
[18]	train-rmse:inf	validation-rmse:inf
[19]	train-rmse:inf	validation-rmse:inf
[20]	train-rmse:inf	validation-rmse:inf
[21]	train-rmse:inf	validation-rmse:inf
[22]	train-rmse:inf	validation-rmse:inf
[23]	train-rmse:inf	validation-rmse:inf
[24]	train-rmse:inf	validation-rmse:inf
[25]	train-rmse:inf	validation-rmse:inf
[26]	train-rmse:inf	validation-rmse:inf
[27]	train-rmse:inf	validation-rmse:inf
[28]	train-rmse:inf	validation-rmse:inf
[29]	train-rmse:inf	validation-rmse:inf
[30]	train-rmse:inf	validation-rmse:in

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:26:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:26:14] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[11]	train-rmse:1074990788849983.75000	validation-rmse:1078040584598995.87500
[12]	train-rmse:9656668027636066.00000	validation-rmse:9683672340785148.00000
[13]	train-rmse:81947611904893632.00000	validation-rmse:82344108348039232.00000
[14]	train-rmse:731643152519285120.00000	validation-rmse:734948349880987392.00000
[15]	train-rmse:4270517686378369024.00000	validation-rmse:4276249013604696576.00000
[16]	train-rmse:inf	validation-rmse:inf
[17]	train-rmse:inf	validation-rmse:inf
[18]	train-rmse:inf	validation-rmse:inf
[19]	train-rmse:inf	validation-rmse:inf
[20]	train-rmse:inf	validation-rmse:inf
[21]	train-rmse:inf	validation-rmse:inf
[22]	train-rmse:inf	validation-rmse:inf
[23]	train-rmse:inf	validation-rmse:inf
[24]	train-rmse:inf	validation-rmse:inf
[25]	train-rmse:inf	validation-rmse:inf
[26]	train-rmse:inf	validation-rmse:inf
[27]	train-rmse:inf	validation-rmse:inf
[28]	train-rmse:inf	validation-rmse:inf
[29]	train-rmse:inf	validation-rmse:inf
[30]	train-rmse:inf	validation-rmse:in

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:26:24] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [20]:


params = {"objective": "reg:squarederror", "tree_method": "gpu_hist", "learning_rate" : 0.01, "max_depth" : 5}

n = 1500

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=25
)

print(f"Best RMSE : {results['test-rmse-mean'].min()}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:40:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best RMSE : 535.1642848700978


Results : https://api.wandb.ai/links/khatgarh-aastha0014/00gvzmr5